In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [133]:
# Read the data
all_data = pd.read_excel("Biopak data compiled final 1-70 v2.xlsx")

# Extract just JVA data
n_columns = len(all_data.columns)
jva_column_count = 14
data = all_data.iloc[:,n_columns-14:].copy()

In [134]:
# Visualize the data
data.head()

,JVA MAP left integral,JVA MAP left Integral <300Hz,JVA MAP left Integral >300Hz,JVA MAP left Integral ratio,JVA MAP left peak amplitude,JVA MAP left peak frequency,JVA MAP left median frequency,JVA MAP right integral,JVA MAP right Integral <300Hz,JVA MAP right Integral >300Hz,JVA MAP right Integral ratio,JVA MAP right peak amplitude,JVA MAP right peak frequency,JVA MAP right median frequency
0,18.2,17.1,1.0,0.06,1.8,24,98,20.6,19.5,1.1,0.06,1.7,20,107
1,21.6,20.4,1.2,0.06,2.0,34,88,36.2,33.5,2.8,0.08,2.5,83,107
2,4.3,3.9,0.4,0.09,0.4,73,83,4.7,4.3,0.4,0.09,0.4,49,88
3,10.1,9.1,1.0,0.11,0.9,78,103,9.7,8.3,1.4,0.17,0.6,78,127
4,8.5,7.4,1.1,0.15,0.6,15,83,76.6,71.0,5.6,0.08,4.4,132,132


In [135]:
def add_quotients(muscle, data):
    intensity_quotients = pd.read_csv(f"quotients/{muscle}/quotient_intensity.csv")
    duration_quotients = pd.read_csv(f"quotients/{muscle}/quotient_duration.csv")
    temp_data_intensity = intensity_quotients.copy()
    temp_data_duration = duration_quotients.copy()
    temp_data_intensity.columns = [f"{muscle}_intensity_" + name
                                for name in temp_data_intensity.columns]
    temp_data_duration.columns = [f"{muscle}_duration_" + name
                                for name in temp_data_duration.columns]
    
    indices_to_insert_nan = [1, 40, 42, 60]

    # Insert NaN values without replacing existing values
    for index in indices_to_insert_nan:
        temp_data_intensity = pd.concat([temp_data_intensity.iloc[:index],
                                        pd.DataFrame([[np.nan] * len(temp_data_intensity.columns)],
                                                    columns=temp_data_intensity.columns),
                                                    temp_data_intensity.iloc[index:]],
                                                    ignore_index=True).copy()
        temp_data_duration = pd.concat([temp_data_duration.iloc[:index],
                                        pd.DataFrame([[np.nan] * len(temp_data_duration.columns)],
                                                    columns=temp_data_duration.columns),
                                                    temp_data_duration.iloc[index:]],
                                                    ignore_index=True).copy()
        
    # Add quotients to JVA data
    new_data = pd.concat([data, temp_data_intensity], axis=1)
    new_data = pd.concat([new_data, temp_data_duration], axis=1)

    return new_data

In [128]:
# Data with quotient
new_data_mmo = add_quotients("mmo", data)
new_data_mle = add_quotients("mle", data)
new_data_map = add_quotients("map", data)


In [129]:
new_data_mmo.head()

,JVA MAP left integral,JVA MAP left Integral <300Hz,JVA MAP left Integral >300Hz,JVA MAP left Integral ratio,JVA MAP left peak amplitude,JVA MAP left peak frequency,JVA MAP left median frequency,JVA MAP right integral,JVA MAP right Integral <300Hz,JVA MAP right Integral >300Hz,...,mmo_intensity_mm_r,mmo_intensity_mm_l,mmo_intensity_da_r,mmo_intensity_da_l,mmo_duration_ta_r,mmo_duration_ta_l,mmo_duration_mm_r,mmo_duration_mm_l,mmo_duration_da_r,mmo_duration_da_l
0,18.2,17.1,1.0,0.06,1.8,24,98,20.6,19.5,1.1,...,0.18,0.19,0.18,0.21,9.47,25.17,73.17,94.00,87.92,137.67
1,21.6,20.4,1.2,0.06,2.0,34,88,36.2,33.5,2.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.3,3.9,0.4,0.09,0.4,73,83,4.7,4.3,0.4,...,0.10,0.10,0.11,0.09,74.83,78.83,87.67,10.00,92.58,9.00
3,10.1,9.1,1.0,0.11,0.9,78,103,9.7,8.3,1.4,...,0.17,0.18,0.18,0.17,16.70,3.50,47.00,67.83,14.75,68.83
4,8.5,7.4,1.1,0.15,0.6,15,83,76.6,71.0,5.6,...,0.12,0.13,0.13,0.12,30.87,17.67,32.83,53.67,28.92,54.67


#### Categorising lateral deviation

In [120]:
# Obtaining necessary columns
closing_max_values = all_data["Deviation max point during closing slant"].copy()
opening_max_values = all_data["Deviation max point during opening slant"].copy()
rom_slant_values = all_data["ROM Slant"]

# Declaring lists to fill in the values according to opening, closing and rom-slant values
deviation_opening_side = []
deviation_closing_side = []

# Fill opening deviation values (yes or no)
for each in opening_max_values:
    if each > 0:
        deviation_opening_side.append("YES")
    else:
        deviation_opening_side.append("NO")

# Fill closing deviation values (yes or no)
for each in closing_max_values:
    if each > 0:
        deviation_closing_side.append("YES")
    else:
        deviation_closing_side.append("NO")

# Fill in lateral deviation values (left, right, 0)
